In [1]:
import numpy as np
import time
from cvxopt import matrix, solvers
considerata=5
C=1.0
read_file = open("ass2data/mnist/train.csv", "r")
lenlinarr=28*28+1
xarr=[]
yarr=[]
numtrain=0
debugcvx=False
trainingtime=0.0
for line in read_file:
    linearr=line.split(',')
    tempyi=int(linearr[lenlinarr-1])
    # print(tempyi)
    if(tempyi==considerata or tempyi==considerata+1):
        # print(tempyi)
        tempxi= []
        for x in range(28*28):
            tempxi.append(1.0*int(linearr[x])/255)
        # print(tempxi)
        xarr.append(tempxi)
        if(tempyi==considerata+1):
            tempyi=1.0
        else:
            tempyi=-1.0
        yarr.append(tempyi)
        # print(xarr)
        numtrain+=1
#         if(numtrain>20):
#             break
xarr = np.array(xarr)
# yarr = np.array(yarr)
# print(xarr[0])
# print(xarr.shape)
# print(yarr.shape)
yarrsq=np.zeros((numtrain,numtrain))
for x in range(numtrain):
    yarrsq[x][x]=yarr[x]
P= np.matmul(np.matmul(yarrsq,np.matmul(xarr, np.transpose(xarr))),yarrsq)
# P=np.zeros((numtrain,numtrain))
# for x in range(numtrain):
#     for y in range(numtrain):
#         P[x][y] = (yarr[x])*(yarr[y])*(np.dot(xarr[x],xarr[y]))
# print("p shape is ")
# print(P.shape)
P=matrix(P)
if(debugcvx): print(P)
q=np.zeros((numtrain,1))
for x in range(numtrain):
    q[x][0] =-1.0
q=matrix(q)
if(debugcvx): print(q)
G=np.zeros((2*numtrain,numtrain))
for x in range(numtrain):
    G[x][x]=-1.0
    G[numtrain+x][x]=1.0
G=matrix(G)
if(debugcvx): print(G)
h=np.zeros((2*numtrain,1))
for x in range(numtrain):
    h[x][0]=0.0
    h[numtrain+x][0]=C
h=matrix(h)
if(debugcvx): print(h)
A=np.zeros((1,numtrain))
for x in range(numtrain):
    A[0][x] = yarr[x]
A=matrix(A)
if(debugcvx): print(A)
# print("A is")
# print(A)
b=matrix(0.0)
if(debugcvx): print(b)
    

starttime = time.time()
sol= solvers.qp(P,q,G,h,A,b)
# print(sol['x'])
alpha=np.array(sol['x'])
xarrsupport=[]
if(debugcvx): print(alpha)
warr=np.zeros((1,28*28))
for x in range(numtrain):
    # print(xarr[x])
    if(alpha[x][0]>pow(10,-4)):
        xarrsupport.append(xarr[x])
    warr = warr + (alpha[x][0])*(yarr[x])*(xarr[x])
    # print(warr)

tempbmax=float('-inf')
tempbmin=float('inf')
for x in range(numtrain):
#     print("warr and xarr shape are")
#     print(warr.shape)
#     print(xarr[x].shape)
    temp = float(np.dot(warr,xarr[x]))
    # print("tempdot is")
    # print(temp)
    if(yarr[x]==-1):
        if(temp>tempbmax):
            tempbmax=temp
    if(yarr[x]==1):
        if(temp<tempbmin):
            tempbmin=temp
bvalue = -(tempbmax+tempbmin)*0.5
endtime=time.time()
print("Training time is")
print(endtime-starttime)
writefile= open("svmbinarysupportvector.txt","w+")
writefile.write(str(xarrsupport))
# print("support vectors are")
# xarrsupport=np.array(xarrsupport)
# print(xarrsupport)
print("number of support vector vs total vectors")
print(len(xarrsupport))
print(numtrain)
writefile= open("svmbinaryw.txt","w+")
writefile.write(str(warr))
# warr=np.transpose(warr)
# print(warr.shape)
print("b is")
print(bvalue)
def findy(xtest):
    temp=0.0
    for x in range(numtrain):
        if(alpha[x][0]==0):
            continue
        temp+= (alpha[x][0])*(yarr[x])*(float(np.dot(xtest,xarr[x])))
    temp+=bvalue
    if(temp>=0):
        return 1.0
    else:
        return -1.0
preddigit=[]
read_file = open("ass2data/mnist/test.csv", "r")
count=0
numtest=0
yarrtest=[]
xarrtest=[]
correct=0
wrong=0
for line in read_file:
    linearr=line.split(',')
    tempyi=int(linearr[lenlinarr-1])
    # print(tempyi)
    if(tempyi==considerata or tempyi==considerata+1):
        # print(tempyi)
        tempxi= []
        for x in range(28*28):
            tempxi.append(1.0*int(linearr[x])/255)
        # print(tempxi)
        xarrtest.append(tempxi)
        if(tempyi==considerata+1):
            tempyi=1.0
        else:
            tempyi=-1.0
        yarrtest.append(tempyi)
        # print(xarr)

        numtest+=1
        temppredy = findy(np.array(tempxi))
        # print("pred is "+str(temppredy))
        # print("real is "+str(tempyi))
        if(temppredy==tempyi):
            correct+=1
        else:
            print(str(wrong)+" wrong in "+str(numtest)+" steps")
            wrong+=1
#         if(numtest>2):
#             break
xarrtest=np.array(xarrtest)
# yarrtest=np.array(yarrtest).reshape(numtest,1)
# yarrnp= np.array(yarr).reshape(numtrain,1)
print("Accuracy is")
print((1.0*correct)/(correct+wrong))



     pcost       dcost       gap    pres   dres
 0: -5.2610e+02 -8.4478e+03  5e+04  3e+00  1e-12
 1: -3.2715e+02 -4.8554e+03  9e+03  4e-01  7e-13
 2: -2.0434e+02 -1.9641e+03  3e+03  1e-01  5e-13
 3: -1.3636e+02 -1.0002e+03  2e+03  6e-02  3e-13
 4: -9.3999e+01 -5.8828e+02  9e+02  3e-02  2e-13
 5: -6.7155e+01 -3.2739e+02  4e+02  1e-02  2e-13
 6: -5.3369e+01 -1.6077e+02  2e+02  5e-03  2e-13
 7: -4.9467e+01 -7.7687e+01  4e+01  8e-04  2e-13
 8: -5.0554e+01 -5.8168e+01  8e+00  8e-15  2e-13
 9: -5.2738e+01 -5.4860e+01  2e+00  1e-14  2e-13
10: -5.3488e+01 -5.3909e+01  4e-01  6e-15  2e-13
11: -5.3665e+01 -5.3686e+01  2e-02  6e-15  2e-13
12: -5.3674e+01 -5.3675e+01  3e-04  4e-16  2e-13
13: -5.3675e+01 -5.3675e+01  3e-06  1e-14  2e-13
Optimal solution found.
Training time is
28.270453453063965
number of support vector vs total vectors
233
4000
b is
-2.407894871839998
0 wrong in 1 steps
1 wrong in 13 steps
2 wrong in 44 steps
3 wrong in 60 steps
4 wrong in 73 steps
5 wrong in 81 steps
6 wrong in 1

In [2]:
gammagaussian=0.05
debuggausssion=False
def findPGaussian():
    if(debuggausssion): print("xarr is")
    if(debuggausssion): print(xarr)
    tempxirow = np.sum(np.multiply(xarr,xarr),axis=1).reshape((numtrain,1))
    if(debuggausssion): print("tempxirow is")
    if(debuggausssion): print(tempxirow)
    tempwhole = tempxirow + np.transpose(tempxirow) 
    if(debuggausssion): print("tempwhole before dot add is")
    if(debuggausssion): print(tempwhole)
    if(debuggausssion): print("dot add is")
    if(debuggausssion): print(((-2)*(np.matmul(xarr,np.transpose(xarr)))))
    tempwhole = tempwhole + ((-2)*(np.matmul(xarr,np.transpose(xarr))))
    if(debuggausssion): print("tempwhole after dot add is")
    if(debuggausssion): print(tempwhole)
    tempwhole = (-gammagaussian)*(tempwhole)
    if(debuggausssion): print("tempwhole for exp is")
    if(debuggausssion): print(tempwhole)
    tempwhole=np.exp(tempwhole)
    tempwhole = np.matmul(np.matmul(yarrsq,tempwhole),yarrsq)
    if(debuggausssion): print(numtrain)
    if(debuggausssion): print(tempwhole.shape)
    return tempwhole
P=matrix(findPGaussian())
# print(P)
starttime=time.time()
sol= solvers.qp(P,q,G,h,A,b)
# print(sol['x'])
alpha=np.array(sol['x'])
if(debugcvx): print(alpha)
# warr=np.zeros((1,28*28))
# for x in range(numtrain):
#     # print(xarr[x])
#     warr = warr + (alpha[x][0])*(yarr[x])*(xarr[x])
#     # print(warr)
# # warr=np.transpose(warr)
# # print(warr)
# supportvectorindex=0
# for x in range(numtrain):
#     if(alpha[x][0]>pow(10,-4)):
#         supportvectorindex=x
#         break
# # bvalue = yarr[supportvectorindex]-float(np.dot(warr,xarr[supportvectorindex]))
# bvalue=0
# def kernel(xi,xj):
#     temp = np.matmul(np.transpose(xi-xj))
debugygaussian=False
def findygaussian(xarrargg,xarrtestargg,numtrainarg,numtestarg,yarrargg,alphaargg):
    temp1 = np.transpose(np.sum(np.multiply(xarrargg,xarrargg),axis=1).reshape((numtrainarg,1)))
    
    if(debugygaussian): print("temp1 is")
    if(debugygaussian): print(temp1)
        
    temp2 = np.sum(np.multiply(xarrtestargg,xarrtestargg),axis=1).reshape((numtestarg,1))
    
    if(debugygaussian): print("temp2 is")
    if(debugygaussian): print(temp2)
    
    temp3 = (-2)*(np.dot(xarrtestargg,np.transpose(xarrargg)))
    
    if(debugygaussian): print("temp3 is")
    if(debugygaussian): print(temp3)
    
    temp4 = temp1+temp2+temp3
    
    if(debugygaussian): print("temp4 is")
    if(debugygaussian): print(temp4)
    
    temp4 = (-gammagaussian)*(temp4)
    temp4 = np.exp(temp4)
    
    if(debugygaussian): print("alphaargg is")
    if(debugygaussian): print(alphaargg)
    if(debugygaussian): print("yarrargg is")
    if(debugygaussian): print(yarrargg)
    
    temp5 = np.multiply(alphaargg,np.array(yarrargg).reshape((numtrainarg,1)))
    
    if(debugygaussian): print("temp5 is =========")
    if(debugygaussian): print(temp5)
#     temp5 = np.diag(np.transpose(temp5)[0])
    
    temp4 = np.matmul(temp4,temp5)
    
    if(debugygaussian): print("prefinal temp4 is")
    if(debugygaussian): print(temp4)
    
    temp4 = temp4
    
    return temp4

# Lets find b
write_file= open("svmbinarysupportgaussian.txt", "w+")

def findbgauss():
    xarrsupport=[]
    yarrsupport=[]
    alphasupport=[]
    numsupport=0
    for x in range(numtrain):
        if(alpha[x][0]>pow(10,-4)):
            xarrsupport.append(xarr[x])
            yarrsupport.append(yarr[x])
            alphasupport.append(alpha[x][0])
            numsupport+=1
#             if(numsupport>1):
#                 break
    print("numsupport vector is "+str(numsupport)+" numtrain is "+str(numtrain))
    write_file.write(str(xarrsupport))
    xarrsupport=np.array(xarrsupport)
    alphasupport=np.transpose(np.array(alphasupport).reshape((1,numsupport)))
    kernalans = findygaussian(xarrsupport,xarrsupport,numsupport,numsupport,yarrsupport,alphasupport)
    yarrsupportnp = np.transpose(np.array(yarrsupport).reshape(1,numsupport))
    allbs = yarrsupportnp-kernalans
    temp=0.0
    for x in range(numsupport):
        temp+=allbs[x][0]
    temp=temp/numsupport
    return temp
bvalue=findbgauss()
endtime= time.time()
print("Training time is")
print(endtime-starttime)
print("b is ")
print(bvalue)
ypredgaussian = findygaussian(xarr,xarrtest,numtrain,numtest,yarr,alpha) + bvalue
debugygaussian2=False
if(debugygaussian2): print("ypredgaussian is")
if(debugygaussian2): print(ypredgaussian)
correct=0
wrong=0
for x in range(numtest):
    if((ypredgaussian[x][0])*(yarrtest[x]) >=0):
#         print("found right for "+str(yarrtest[x]) + "val was "+str(ypredgaussian[x][0]))
        correct+=1
    else:
#         print("found wrong for "+str(yarrtest[x]) + "val was "+str(ypredgaussian[x][0]))
        wrong+=1
print("Accuracy is")
print((1.0*correct)/(correct+wrong))

     pcost       dcost       gap    pres   dres
 0: -1.9096e+02 -5.7730e+03  2e+04  2e+00  2e-15
 1: -1.1383e+02 -2.7535e+03  4e+03  1e-01  1e-15
 2: -1.6306e+02 -5.4871e+02  4e+02  1e-02  2e-15
 3: -2.0648e+02 -2.9981e+02  1e+02  2e-03  2e-15
 4: -2.1929e+02 -2.4903e+02  3e+01  5e-04  1e-15
 5: -2.2479e+02 -2.3257e+02  8e+00  6e-05  1e-15
 6: -2.2651e+02 -2.2819e+02  2e+00  1e-05  1e-15
 7: -2.2700e+02 -2.2713e+02  1e-01  5e-07  1e-15
 8: -2.2704e+02 -2.2705e+02  5e-03  1e-08  1e-15
 9: -2.2705e+02 -2.2705e+02  1e-04  2e-10  1e-15
Optimal solution found.
numsupport vector is 1496 numtrain is 4000
Training time is
26.477439165115356
b is 
-0.13989252271787814
Accuracy is
0.9918918918918919


In [3]:
from svmutil import *
# svm_model.predict = lambda self, x: svm_predict([0], [x], self)[0][0]

prob = svm_problem(yarr, xarr)

param = svm_parameter('-s 0 -c 1.0 -t 0')
# param = svm_parameter('-s 0 -c 1.0 -t 2 -g 0.05')

starttime=time.time()
m=svm_train(prob, param)
endtime=time.time()
print("Traintime is")
print(endtime-starttime)
# print("nsv")
# print(len(m.nSV))
# print(type(m.nSV))

predy = svm_predict(yarrtest,xarrtest,m)

# param = svm_parameter('-s 0 -c 1.0 -t 0')
param = svm_parameter('-s 0 -c 1.0 -t 2 -g 0.05')

starttime=time.time()

m=svm_train(prob, param)

endtime=time.time()
print("Traintime is")
print(endtime-starttime)
# print("nsv")
# print(m.nSV)
# print(type(m.nSV))

predy = svm_predict(yarrtest,xarrtest,m)

# m.predict(xarrtest[0])
# correct=0
# wrong=0
# for x in range(numtest):
#     predy = m.predict(xarrtest[x])
#     if(predy==yarrtest[x]):
#         correct+=1
#     else:
#         wrong+=1

# print("Accuracy is")
# print((1.0*correct)/(correct+wrong))

Traintime is
1.2618520259857178
Accuracy = 97.2973% (1800/1850) (classification)
Traintime is
5.1232078075408936
Accuracy = 99.1892% (1835/1850) (classification)
